# 模块中的增删改查等方法
### 增：
- create(self, vals)
    - vals是一个字段，key是字段名字，value是字段值
    - res是新建记录的记录集，create方法创建成功后，返回值为新的记录集
    - ORM操作：self.create({'key': value})
            @api.model
            def create(self, vals):
                res = super(ClassName, self).create(vals)
                return res

### 删
- unlink方法
    - 删除当前记录集的所有记录，返回结果是一个布尔值
    - 可以在这个位置重写unlink方法来实现伪删除
    - ORM操作：self.unlink()
            @api.multi
            def unlink(self):
                res = super(ProjectProject, self).unlink()
                return res
                
### 改
- write方法
            @api.multi
            def _write(self, vals):
                old_note = self.note
                res = super(EpidemicRecord, self)._write(vals)
                new_note = self.note
                return res
    - vals是一个字典，里面存储了多组键值对
    - ORM操作：self.write({'key': value})
    - 记录集在调用write方法时，只能逐条修改，如果记录集有多条记录，需要遍历
    - 字典值有三种格式：一种Many2many字段，一种One2many,另一种是其他字段
        - Many2many值表现形式
            - (0,0,{values}) 根据values里面的信息新建一个记录。

            - (1,ID,{values})更新id=ID的记录（写入values里面的数据）

            - (2,ID) 删除id=ID的数据（调用unlink方法，删除数据以及整个主从数据链接关系）

            - (3,ID) 切断主从数据的链接关系但是不删除这个数据

            - (4,ID) 为id=ID的数据添加主从链接关系。

            - (5) 删除所有的从数据的链接关系就是向所有的从数据调用(3,ID)

            - (6,0,[IDs]) 用IDs里面的记录替换原来的记录（就是先执行(5)再执行循环IDs执行（4,ID））

                    例子[(6, 0, [8, 5, 6, 4])] 设置 many2many to ids [8, 5, 6, 4]
        - 其他字段：
            - id或字符
        - One2Many在vals中值的表现形式
            - 新增：(0, _, values)，values类型为dict
            - 删除： (2, id, _)：删除这个id对应的记录（从数据库中删除），不能在create中使用
            - 更新: (1, id, values)：更新当前id对应的记录字段的值，values类型为dict，不能在create中使用
            - 移除：(3, id, _):从集合中移除ID为id的记录，但不从数据库中删除它，不能在create中使用
            - 添加到集合：(4, id, _):将已存在的ID为id的记录添加到集合
            - 删除所有集合记录：(5, _, _):从集合中删除所有记录，相当于在每个记录上显式使用命令3，不能再create中使用, 不能在One2Many中使用
            - (6, _, ids):移除当前的所有记录，添加ids所对应的记录，相当于先执行5再执行4

### 查
- browse方法
    - self.browse(ids)方法，ids是一个列表
- search方法
    - Model.search(args[, offset=0][, limit=None][, order=None][, count=False])
        - domain是查询条件。limit限制取前几个，order排序，offset需要忽略几个记录，count如果为True返回查到的个数
- search_count
    - 直接返回查找到的个数
- Model.name_search(name='', args=None, operator='ilike', limit=100) → records
    - 通过display_name查找
- read
    - 低级rpc的调用，python中使用browse
- read_group对查到的结果进行分组呈现
    - Model.read_group(domain, fields, groupby, offset=0, limit=None, orderby=False, lazy=True)

### 其他方法   
- copy:复制：
    - copy(default=None):default为dict类型，表示要更改的值
- default_get:获取字段默认值
    - default_get(fields_list)
    - 只有当这个字段的默认值是requed的时候才能获取到。没要求的没必要获取
- name_create:仅通过name创建新记录
    - name_create(name)
    - 返回类型是元组类型
    - name_get()得到已经创建记录
- flush:处理所有挂载的记录到数据库
    - Model.flush(fnames=None, records=None)
- fields_get:得到每个字段的定义
    - Model.fields_get([fields][, attributes])
- fields_view_get：得到每个视图的定义
    - Model.fields_view_get([view_id | view_type='form'])
    
### 记录或记录集操作
- ids 得到当前记录集合的id列表
- ensure_one() 验证一个记录集合是否只包含一条记录
- exists() 返回当前记录集中真正存在的子集，并把缓存中未删除的部分做标记，可用于判断if record.exists():
- name_get()：返回（id,display_name)
- get_metadata():得到记录的元数据：有id,create_uid,create_date,write_uid,write_date,xmlid,noupdate
- filtered(func) 返回满足func参数内条件的记录集合，参数可以是一个函数或者用.分隔的字段列表
- filtered_domain(domain)
- sorted(key=None, reverse=False) 返回按key排序之后的记录集，key参数可以是一个返回单个key的函数或字段名称或为空，reverse参数为True时即为倒序
- mapped(arg) 将func函数应用到所有记录上，并返回记录列表或集合,arg可以为func，filed_name,field.field
- 集合之间的计算操作：
    - in , not in 
    - <=，<: A <= B,A是否是B的子集
    - >=, >
    - |:返回并集
    - &：交集
    - -： A - B：表示在A中但是在B中没有的：